In [1]:
from csv import DictWriter
from lxml import etree
from os import listdir
from os.path import isfile, join
from tqdm import tqdm

In [2]:
inputFolder = '../data/source/multimedia/'
outputFile = 'output/correspondences.csv'

In [3]:
files = [f for f in listdir(inputFolder) if isfile(join(inputFolder, f)) and f.endswith('.xml')]

In [4]:
results = []

In [5]:
for file in tqdm(files):
    tree = etree.parse(join(inputFolder, file))
    uuid = tree.find('.//{http://www.zetcom.com/ria/ws/module}moduleItem').get('uuid')
    filenameElement = tree.find('.//{http://www.zetcom.com/ria/ws/module}dataField[@name="MulOriginalFileTxt"]/{http://www.zetcom.com/ria/ws/module}value')
    if filenameElement is not None:
        filename = filenameElement.text
        results.append({
            'uuid': uuid,
            'filename': filename
        })

100%|██████████| 299371/299371 [01:05<00:00, 4554.17it/s]


In [6]:
results = sorted(results, key=lambda x: x['uuid'])

In [7]:
with open(outputFile, 'w') as f:
    writer = DictWriter(f, fieldnames=('uuid' ,'filename'))
    writer.writeheader()
    for row in results:
        writer.writerow(row)